# Importing Libraries

In [2]:
! pip install psutil -q
! pip install pandarallel -q
! pip install -U sentence-transformers -q
! pip install annoy -q

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 2.8 MB 18.4 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 895 kB 53.2 MB/s 
     |████████████████████████████████| 636 kB 68.1 MB/s 
     |████████████████████████████████| 3.3 MB 37.8 MB/s 
     |████████████████████████████████| 646 kB 5.3 MB/s 


In [3]:
import psutil

workers = psutil.cpu_count()
# psutil.cpu_count(logical=False)

In [4]:
from pandarallel.utils import progress_bars
progress_bars.is_notebook_lab = lambda : True

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=workers, use_memory_fs=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
from sentence_transformers import SentenceTransformer ,  util
import torch
import pandas as pd
import numpy as np
import string
import re
from annoy import AnnoyIndex
from tqdm.auto import tqdm
tqdm.pandas()

import pickle

# Generate embeddings and save

In [ ]:
# Import the sentence transformer model
model = SentenceTransformer('ghanashyamvtatti/roberta-fake-news', device = 'cuda')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ghanashyamvtatti_roberta-fake-news were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/ghanashyamvtatti_roberta-fake-news and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN

In [ ]:
def lower(text):
  return text.lower()

# remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  # text = text.translate(str.maketrans('', ' ', string.punctuation))
  # return text

  res = re.sub(r'[^\w\s]', '', text)
  return res

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

# remove multiple spaces
def remove_mul_space(text):
  return " ".join(text.split())

def clean(text):
  
  text = lower(text)
  text = remove_urls(text)
  text = remove_nonascii(text)
  text = remove_punctuations(text)
  text = remove_num(text)
  text = remove_mul_space(text)

  return text

def generate_embeddings(sentences):
  
  embeddings = model.encode(sentences=sentences, show_progress_bar=True)
  embeddings=[torch.from_numpy(item) for item in embeddings]
  
  return embeddings

def convert_to_array(tensor):
  return tensor.detach().cpu().numpy()

def get_tokens(text):
  return len(re.findall(r'\w+', text))

In [ ]:
df_true = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Kaggle/True.csv")
df_fake = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Kaggle/Fake.csv")

In [ ]:
df_true['total_text'] = df_true['title'].fillna('') + " " + df_true['text'].fillna('')
df_fake['total_text'] = df_fake['title'].fillna('') + " " + df_fake['text'].fillna('')

In [ ]:
df_true['total_text'] = df_true['total_text'].parallel_apply(clean)
df_fake['total_text'] = df_fake['total_text'].parallel_apply(clean)

In [ ]:
df_true['label'] = 'True'
df_fake['label'] = 'Fake'

frames = [df_true, df_fake]
df = pd.concat(frames, ignore_index=True)

In [ ]:
statements = df['total_text'].tolist()
embeddings = generate_embeddings(statements)
lengths = df['total_text'].parallel_apply(get_tokens)
df['embedding'] = embeddings
df['embedding'] = df['embedding'].parallel_apply(convert_to_array)

Batches:   0%|          | 0/1404 [00:00<?, ?it/s]

In [ ]:
# df.to_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/ghanashyamvtatti roberta-fake-news/df_kaggle_with_embeddings.pickle")

# Analysis

In [13]:
n = 500

f = 768
ann = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
ann.load(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/ghanashyamvtatti roberta-fake-news/annoy/embeddings_"+str(n)+".ann")

embeddings_df = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/ghanashyamvtatti roberta-fake-news/embeddings/df_word_embeddings_"+str(n)+".pickle")

In [14]:
embeddings_df[['word', 'common_score', 'true_score', 'fake_score']]

,word,common_score,true_score,fake_score
0,kurdish,0.000000,55.096345,0.000000
1,message,0.000000,0.000000,53.483613
2,further,56.012499,64.714631,0.000000
3,through,125.528925,118.760238,126.040170
4,thing,145.151220,0.000000,213.094031
...,...,...,...,...
570,nation,140.005647,103.915208,94.500946
571,news,270.129748,164.723252,349.942980
572,staff,56.652495,64.660747,0.000000
573,beijing,0.000000,56.254850,0.000000


In [15]:
words_574 = embeddings_df[['word', 'common_score', 'true_score', 'fake_score']]

words_574.to_csv(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/ghanashyamvtatti roberta-fake-news/wordlists/words_574.csv', index=False)

In [12]:
df = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/ghanashyamvtatti roberta-fake-news/df_kaggle_with_embeddings.pickle")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       44898 non-null  object
 1   text        44898 non-null  object
 2   subject     44898 non-null  object
 3   date        44898 non-null  object
 4   total_text  44898 non-null  object
 5   label       44898 non-null  object
 6   embedding   44898 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


In [14]:
def get_score(embedding, n):

  fake_score, true_score, common_score = 0.0, 0.0, 0.0
  indexes, distances = ann.get_nns_by_vector(embedding, n, include_distances=True, search_k=-1)

  for j in range(len(indexes)):
    fake_score += embeddings_df['fake_score'][indexes[j]] / (distances[j])
    true_score += embeddings_df['true_score'][indexes[j]] / (distances[j])
    common_score += embeddings_df['common_score'][indexes[j]] / (distances[j])

  return [fake_score, true_score, common_score]

def get_fake_score(embedding):
  fake_score = 0.0
  indexes, distances = ann.get_nns_by_vector(embedding, n, include_distances=True, search_k=-1)

  for j in range(len(indexes)):
    fake_score += embeddings_df['fake_score'][indexes[j]] / (distances[j])
  
  return fake_score

def get_true_score(embedding):
  true_score = 0.0
  indexes, distances = ann.get_nns_by_vector(embedding, n, include_distances=True, search_k=-1)

  for j in range(len(indexes)):
    true_score += embeddings_df['true_score'][indexes[j]] / (distances[j])
  
  return true_score

def get_common_score(embedding):
  common_score = 0.0
  indexes, distances = ann.get_nns_by_vector(embedding, n, include_distances=True, search_k=-1)

  for j in range(len(indexes)):
    common_score += embeddings_df['common_score'][indexes[j]] / (distances[j])
  
  return common_score

def get_length(text):
  return len(re.findall(r'\w+', text))

def get_label(result):
  label = ['Fake', 'True']
  max_value = max(result[:2])

  return label[result.index(max_value)]

In [23]:
%%time

df['length'] = df['total_text'].parallel_apply(get_length)
df['scores'] = df['embedding'].progress_apply(get_score, n=1000)

  0%|          | 0/44898 [00:00<?, ?it/s]

CPU times: user 1h 2min 48s, sys: 55.3 s, total: 1h 3min 43s
Wall time: 1h 3min 26s


In [24]:
df['prediction'] = df['scores'].parallel_apply(get_label)

In [25]:
from sklearn.metrics import accuracy_score

y_pred = df['prediction'].tolist()
y_true = df['label'].tolist()
accuracy_score(y_true, y_pred)

0.7289634282150652

In [26]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.94761722, 0.48923752])